<a href="https://colab.research.google.com/github/sayyadaliya90-sketch/Air-Quality-Index/blob/main/A_10_Aliya_Sayyad_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Step 1: Install required libraries
!pip install qiskit qiskit-aer

# Step 2: Import libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer   # simulators
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import MCXGate
import matplotlib.pyplot as plt

# Step 3: Define the Oracle (searching for state |11>)
oracle = QuantumCircuit(2)
oracle.cz(0, 1)  # Mark state |11>

# Step 4: Define the Diffuser (inversion about the mean)
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    qc.h(range(nqubits))
    qc.x(range(nqubits))
    qc.h(nqubits-1)
    qc.append(MCXGate(nqubits-1), list(range(nqubits)))  # multi-controlled X
    qc.h(nqubits-1)
    qc.x(range(nqubits))
    qc.h(range(nqubits))
    return qc

# Step 5: Build Grover’s Algorithm circuit
n = 2
grover_circuit = QuantumCircuit(n, n)

# Initialize into superposition
grover_circuit.h(range(n))

# Apply Grover iteration (Oracle + Diffuser)
grover_circuit.append(oracle, range(n))
grover_circuit.append(diffuser(n), range(n))

# Measurement
grover_circuit.measure(range(n), range(n))

# Show circuit diagram (safe text mode)
print(grover_circuit.draw('text'))

# Step 6: Run on simulator
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(grover_circuit, simulator)
job = simulator.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts()

# Step 7: Show results
print("Measurement Results:", counts)
plot_histogram(counts)
plt.show()


     ┌───┐┌─────────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0            ├┤0            ├┤M├───
     ├───┤│  circuit-59 ││  circuit-63 │└╥┘┌─┐
q_1: ┤ H ├┤1            ├┤1            ├─╫─┤M├
     └───┘└─────────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════════╩══╩═
                                         0  1 
Measurement Results: {'11': 1024}


In [59]:
# qft_demo.py
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector

def qft_circuit(n):
    qc = QuantumCircuit(n)
    for j in range(n):
        qc.h(j)
        # apply controlled rotations
        for k in range(2, n-j+1):
            angle = np.pi / (2**(k-1))
            qc.cp(angle, j+k-1, j)  # control is higher index for convention
    # Finally reverse qubits (bit reversal)
    qc = qc.swap_bits(range(n))
    return qc

def inverse_qft_circuit(n):
    return qft_circuit(n).inverse()

# Helper: swap_bits implementation (QuantumCircuit has no direct swap_bits in earlier versions)
def add_swap_bits(qc, qubits):
    n = len(qubits)
    for i in range(n//2):
        qc.swap(qubits[i], qubits[n-1-i])

# We'll build the QFT carefully because cp signature differs by qiskit version. Use controlled rotation manually:
def qft(n):
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.h(i)
        for j in range(i+1, n):
            lam = np.pi / (2**(j-i))
            # apply controlled phase with control j and target i: use cp if available otherwise use cu1
            try:
                qc.cp(lam, j, i)   # newer Qiskit
            except Exception:
                qc.cu1(lam, j, i)  # fall back
    # reverse order
    for i in range(n//2):
        qc.swap(i, n-1-i)
    return qc

# Test QFT on basis state |1>
if __name__ == "__main__":
    n = 3
    qc = QuantumCircuit(n)
    qc.x(0)  # prepare |1> (LSB at index 0)
    qc.append(qft(n), range(n))
    sv = Statevector.from_instruction(qc)
    probs = sv.probabilities_dict()
    print("Statevector probabilities after QFT on |1> (3 qubits):")
    print(probs)
    print("\nCircuit:")
    print(qc.draw(output='text'))


Statevector probabilities after QFT on |1> (3 qubits):
{np.str_('000'): np.float64(0.12499999999999994), np.str_('001'): np.float64(0.12499999999999994), np.str_('010'): np.float64(0.12499999999999994), np.str_('011'): np.float64(0.12499999999999994), np.str_('100'): np.float64(0.12499999999999994), np.str_('101'): np.float64(0.12499999999999994), np.str_('110'): np.float64(0.12499999999999994), np.str_('111'): np.float64(0.12499999999999994)}

Circuit:
     ┌───┐┌──────────────┐
q_0: ┤ X ├┤0             ├
     └───┘│              │
q_1: ─────┤1 circuit-167 ├
          │              │
q_2: ─────┤2             ├
          └──────────────┘


In [60]:
pip install qiskit qiskit-aer numpy scipy


In [62]:
# qft_demo.py
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector

def qft_circuit(n):
    qc = QuantumCircuit(n)
    for j in range(n):
        qc.h(j)
        # apply controlled rotations
        for k in range(2, n-j+1):
            angle = np.pi / (2**(k-1))
            qc.cp(angle, j+k-1, j)  # control is higher index for convention
    # Finally reverse qubits (bit reversal)
    qc = qc.swap_bits(range(n))
    return qc

def inverse_qft_circuit(n):
    return qft_circuit(n).inverse()

# Helper: swap_bits implementation (QuantumCircuit has no direct swap_bits in earlier versions)
def add_swap_bits(qc, qubits):
    n = len(qubits)
    for i in range(n//2):
        qc.swap(qubits[i], qubits[n-1-i])

# We'll build the QFT carefully because cp signature differs by qiskit version. Use controlled rotation manually:
def qft(n):
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.h(i)
        for j in range(i+1, n):
            lam = np.pi / (2**(j-i))
            # apply controlled phase with control j and target i: use cp if available otherwise use cu1
            try:
                qc.cp(lam, j, i)   # newer Qiskit
            except Exception:
                qc.cu1(lam, j, i)  # fall back
    # reverse order
    for i in range(n//2):
        qc.swap(i, n-1-i)
    return qc

# Test QFT on basis state |1>
if __name__ == "__main__":
    n = 3
    qc = QuantumCircuit(n)
    qc.x(0)  # prepare |1> (LSB at index 0)
    qc.append(qft(n), range(n))
    sv = Statevector.from_instruction(qc)
    probs = sv.probabilities_dict()
    print("Statevector probabilities after QFT on |1> (3 qubits):")
    print(probs)
    print("\nCircuit:")
    print(qc.draw(output='text'))


Statevector probabilities after QFT on |1> (3 qubits):
{np.str_('000'): np.float64(0.12499999999999994), np.str_('001'): np.float64(0.12499999999999994), np.str_('010'): np.float64(0.12499999999999994), np.str_('011'): np.float64(0.12499999999999994), np.str_('100'): np.float64(0.12499999999999994), np.str_('101'): np.float64(0.12499999999999994), np.str_('110'): np.float64(0.12499999999999994), np.str_('111'): np.float64(0.12499999999999994)}

Circuit:
     ┌───┐┌──────────────┐
q_0: ┤ X ├┤0             ├
     └───┘│              │
q_1: ─────┤1 circuit-173 ├
          │              │
q_2: ─────┤2             ├
          └──────────────┘


In [64]:
# Install required packages
!pip install qiskit qiskit-aer

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

# Function to create QFT circuit
def qft_circuit(n):
    qc = QuantumCircuit(n)
    for j in range(n):
        qc.h(j)
        for k in range(j+1, n):
            qc.cp(np.pi/2**(k-j), k, j)
    # Swap qubits (reverse order)
    for i in range(n//2):
        qc.swap(i, n-i-1)
    return qc

# Example: 3-qubit QFT
n = 3
qft = qft_circuit(n)
print(qft.draw('text'))

# Run QFT on |1> state
qc = QuantumCircuit(n, n)
qc.x(0)  # prepare state |001>
qc.append(qft, range(n))
qc.measure(range(n), range(n))

# Simulate
sim = Aer.get_backend('qasm_simulator')
compiled = transpile(qc, sim)
result = sim.run(compiled, shots=1024).result()
counts = result.get_counts()

print("QFT Output:", counts)
plot_histogram(counts)
plt.show()


     ┌───┐                                        
q_0: ┤ H ├─■────────■───────────────────────────X─
     └───┘ │P(π/2)  │       ┌───┐               │ 
q_1: ──────■────────┼───────┤ H ├─■─────────────┼─
                    │P(π/4) └───┘ │P(π/2) ┌───┐ │ 
q_2: ───────────────■─────────────■───────┤ H ├─X─
                                          └───┘   
QFT Output: {'110': 110, '001': 128, '101': 124, '111': 130, '011': 125, '100': 137, '010': 120, '000': 150}


In [67]:
# Install required packages
!pip install qiskit qiskit-aer

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT  # Built-in QFT
import numpy as np
import math
import random
from fractions import Fraction

# Helper: modular exponentiation as a quantum circuit
def modexp(a, N, power, n_count):
    """
    Build the circuit that does: |x> |1> -> |x> |a^x mod N>
    Here x is in the counting register (n_count qubits),
    the second register (target) has enough qubits to hold N.
    We will use repeated squaring technique.
    For demonstration only; not optimized.
    """
    # number of bits to represent N
    n_target = N.bit_length()
    qc = QuantumCircuit(n_count + n_target)
    # We'll do a controlled-multiplication by a^(2^i) mod N
    for i in range(n_count):
        # compute exponent = 2^i
        exp = pow(a, 2**i, N)
        # controlled multiply by exp mod N
        qc.cp(2 * np.pi * 0 / 1, i, n_count)  # dummy, no-op placeholder
        # For simplicity, I'm not implementing full modular multiplication here
        # Real circuits require ancilla, controlled multiplication, modular reduction, etc.
    return qc

def qpe_modexp(a, N, n_count):
    """ Quantum Phase Estimation circuit for modular exponentiation """
    # Number of target qubits
    n_target = N.bit_length()
    qc = QuantumCircuit(n_count + n_target, n_count)

    # 1) Put counting register in superposition
    qc.h(range(n_count))

    # 2) Prepare target in |1>
    qc.x(n_count + n_target - 1)

    # 3) Apply controlled-U^{2^j} operations
    # where U acts as multiplication by a mod N
    for j in range(n_count):
        # Controlled modular exponentiation by a^(2^j)
        qc.append(modexp(a, N, 2**j, n_count),
                  list(range(j, j+1)) + list(range(n_count, n_count + n_target)))

    # 4) Apply inverse QFT on counting qubits
    qc.append(QFT(n_count, inverse=True, do_swaps=False), range(n_count))

    # 5) Measure counting register
    qc.measure(range(n_count), range(n_count))

    return qc

def find_order(a, N):
    """ Use QPE to find order r such that a^r mod N = 1 """
    # choose number of counting qubits (precision)
    n_count = 4  # you may increase for better precision
    qc = qpe_modexp(a, N, n_count)
    sim = Aer.get_backend('qasm_simulator')
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=1024).result()
    counts = result.get_counts()
    print("QPE counts:", counts)
    # pick highest result
    measured = max(counts, key=counts.get)
    phase = int(measured, 2) / (2**n_count)
    # estimate r via continued fractions
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    return r

def shor(N):
    if N % 2 == 0:
        return 2
    # pick a random a in [2, N-2]
    a = random.randrange(2, N-1)
    g = math.gcd(a, N)
    if g > 1:
        return g
    # find order r
    r = find_order(a, N)
    print("Found r =", r)
    if r % 2 != 0:
        return None  # try again
    # compute factors
    x = pow(a, r//2, N)
    if x == N - 1:
        return None  # failure, try different a
    p = math.gcd(x + 1, N)
    q = math.gcd(x - 1, N)
    if p * q == N:
        return (p, q)
    else:
        return None

# Try factoring N = 15
N = 15
factors = None
for _ in range(3):
    f = shor(N)
    if f and f != None:
        factors = f
        break

print("Factors of", N, "=", factors)


Factors of 15 = 5
